In [1]:
!pip install pyModeS

     |████████████████████████████████| 58 kB 5.5 MB/s 
  Created wheel for pyModeS: filename=pyModeS-2.9-cp37-cp37m-linux_x86_64.whl size=360525 sha256=67ef63eada1edec6fd66e8019bff1022732e34ae9c63509f355b18c3f0331360
  Stored in directory: /root/.cache/pip/wheels/86/8a/a6/e7fac1d303e55514a137ee719b6e8fbc5e9bff0f929577b0cd
Successfully built pyModeS


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive"

In [ ]:
import pyModeS as pms
import pandas as pd
from pandas import ExcelWriter
import csv

mensajes = "./gdrive/My Drive/TFG/ArchivosAsterix/ADSB_HEX_08.txt"


############# Funciones ################

def leerCategoria(hex):
  if hex[0:2] == '15':
    return True

def leerLongitud(hex):
  global longitud
  longitud = pms.hex2int(hex[2:6])
  LONG.append(longitud)

def leerFspec(hex):
  global fspec_hex, fspec_h, fspec_bin, fspec_bin2, fspec_b, len_fspec
  contador = list(range(20))
  for i in contador:
    campo_hex = hex[6+i*2:8+i*2]
    fspec_hex += campo_hex
    campo_bin = pms.hex2bin(campo_hex)
    fspec_b = fspec_b + campo_bin[:-1]
    if campo_bin[-1] != '1':
      break
    else:
      continue
  fspec_bin = pms.hex2bin(fspec_hex)
  FSPEC_HEX.append(fspec_hex)
  FSPEC_BIN.append(fspec_bin)
  FSPEC_B.append(fspec_b)

  # fspec_b = fspec_bin
  # for i in list(range(len(fspec_bin))):
  #   if (i+1) & 8:
  #     fspec_b = fspec_b[:i] + fspec_b[i+1:]

  #   print(fspec_b)
  # FSPEC_B.append(fspec_b)


def analizarFspec(hex,fspec_hex,fspec_bin):
  #vercampos(hex,fspec)
  len_fspec = list(range(len(fspec_bin)))
  lugar = len(fspec_hex) + 6
  for i in len_fspec:
    if fspec_bin[6] == '1' and fspec_bin[10] == '1' and fspec_bin[11] == '1' and fspec_bin[15] == '1' and fspec_bin[28] == '1' and fspec_bin[29] == '1':
      if fspec_bin[i] == '1':
        campo = i + 1
        if campo == 1:
          #print(f'I010: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 2:
          contador = list(range(10))
          for i in contador:
            campo_hex = hex[lugar:lugar+2]
            campo_bin = pms.hex2bin(campo_hex)
            if campo_bin[-1] != '1':
              lugar += 2
              break
            else:
              lugar += 2
              continue
        if campo == 3:
          #print(f'I161: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 4:
          #print(f'I015: {hex[lugar:lugar+2]}')
          lugar += 2
        if campo == 5: 
          #print(f'I071: {hex[lugar:lugar+6]}')
          lugar += 6
        if campo == 6:
          msg = hex[lugar:lugar+12]
          #print(f'I130: {msg}')
          lugar += 12
        if campo == 7:
          msg = hex[lugar:lugar+16]
          s1 = msg[:len(msg)//2]
          s2 = msg[len(msg)//2:]
          lat = int(s1, 16)*(180/(2**30))
          lon = int(s2, 16)*(180/(2**30))
          if lon > 360:
            lon = lon - 360
          LAT.append(lat)
          LON.append(lon)
          #print(f'I131: Lat-> {lat}, Lon-> {lon}')
          lugar += 16
        if campo == 8:
          #print(f'I072: {hex[lugar:lugar+6]}')
          lugar += 6
        if campo == 9:
          #print(f'I150: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 10:
          #print(f'I151: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 11:
          t_addr = hex[lugar:lugar+6]
          T_ADDR.append(t_addr)
          #print(f'I080: {t_addr}')
          lugar += 6
        if campo == 12:
          msg = hex[lugar:lugar+6]
          time = int(msg, 16)*(1/128)
          TIME.append(time)
          #print(f'I073: {time}')
          lugar += 6
        if campo == 13:
          #print(f'I074: {hex[lugar:lugar+8]}')
          lugar += 8
        if campo == 14:
          #print(f'I075: {hex[lugar:lugar+6]}')
          lugar += 6
        if campo == 15:
          #print(f'I076: {hex[lugar:lugar+8]}')
          lugar += 8
        if campo == 16:
          msg = hex[lugar:lugar+4]
          h = int(msg, 16)*6.25
          HEIGHT.append(h)
          #print(f'I140: {h}')
          lugar += 4
        if campo == 17:
          contador = list(range(10))
          for i in contador:
            campo_hex = hex[lugar:lugar+2]
            campo_bin = pms.hex2bin(campo_hex)
            if campo_bin[-1] != '1':
              lugar += 2
              break
            else:
              lugar += 2
              continue 
        if campo == 18:
          #print(f'I210: {hex[lugar:lugar+2]}')
          lugar += 2
        if campo == 19:
          #print(f'I070: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 20:
          #print(f'I230: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 21:
          #print(f'I145: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 22:
          #print(f'I152: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 23:
          #print(f'I200: {hex[lugar:lugar+2]}')
          lugar += 2
        if campo == 24:
          #print(f'I155: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 25:
          #print(f'I157: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 26:
          #print(f'I160: {hex[lugar:lugar+8]}')
          lugar += 8
        if campo == 27:
          #print(f'I165: {hex[lugar:lugar+4]}')
          lugar += 4
        if campo == 28:
          #print(f'I077: {hex[lugar:lugar+6]}')
          lugar += 6
        if campo == 29:
          msg = pms.hex2bin(hex[lugar:lugar+12])
          s1 = msg[:len(msg)//8]
          s2 = msg[len(msg)//8:2*len(msg)//8]
          s3 = msg[2*len(msg)//8:3*len(msg)//8]
          s4 = msg[3*len(msg)//8:4*len(msg)//8]
          s5 = msg[4*len(msg)//8:5*len(msg)//8]
          s6 = msg[5*len(msg)//8:6*len(msg)//8]
          s7 = msg[6*len(msg)//8:7*len(msg)//8]
          s8 = msg[7*len(msg)//8:]
          s = [s1,s2,s3,s4,s5,s6,s7,s8]
          t_id = list(range(len(s)))
          for i in list(range(len(s))):
            if s[i] == '000001':
              t_id[i] = 'A'
            elif s[i] == '000010':
              t_id[i] = 'B'
            elif s[i] == '000011':
              t_id[i] = 'C'
            elif s[i] == '000100':
              t_id[i] = 'D'
            elif s[i] == '000101':
              t_id[i] = 'E'
            elif s[i] == '000110':
              t_id[i] = 'F'
            elif s[i] == '000111':
              t_id[i] = 'G'
            elif s[i] == '001000':
              t_id[i] = 'H'
            elif s[i] == '001001':
              t_id[i] = 'I'
            elif s[i] == '001010':
              t_id[i] = 'J'
            elif s[i] == '001011':
              t_id[i] = 'K'
            elif s[i] == '001100':
              t_id[i] = 'L'
            elif s[i] == '001101':
              t_id[i] = 'M'
            elif s[i] == '001110':
              t_id[i] = 'N'
            elif s[i] == '001111':
              t_id[i] = 'O'
            elif s[i] == '010000':
              t_id[i] = 'P'
            elif s[i] == '010001':
              t_id[i] = 'Q'
            elif s[i] == '010010':
              t_id[i] = 'R'
            elif s[i] == '010011':
              t_id[i] = 'S'
            elif s[i] == '010100':
              t_id[i] = 'T'
            elif s[i] == '010101':
              t_id[i] = 'U'
            elif s[i] == '010110':
              t_id[i] = 'V'
            elif s[i] == '010111':
              t_id[i] = 'W'
            elif s[i] == '011000':
              t_id[i] = 'X'
            elif s[i] == '011001':
              t_id[i] = 'Y'
            elif s[i] == '011010':
              t_id[i] = 'Z'
            elif s[i] == '100000':
              t_id[i] = ''
            elif s[i] == '110000':
              t_id[i] = '0'
            elif s[i] == '110001':
              t_id[i] = '1'
            elif s[i] == '110010':
              t_id[i] = '2'
            elif s[i] == '110011':
              t_id[i] = '3'
            elif s[i] == '110100':
              t_id[i] = '4'
            elif s[i] == '110101':
              t_id[i] = '5'
            elif s[i] == '110110':
              t_id[i] = '6'
            elif s[i] == '110111':
              t_id[i] = '7'
            elif s[i] == '111000':
              t_id[i] = '8'
            elif s[i] == '111001':
              t_id[i] = '9'
          t_id2 = "".join(map(str,t_id))
          T_ID.append(t_id2)
          #print(f'I170: {t_id2}')
          lugar += 12
        if campo == 30:
          msg = int(hex[lugar:lugar+2], 16)
          if msg == 0:
            e_cat = 'No Info.'
          elif msg == 1:
            e_cat = 'Light Aircraft'
          elif msg == 2:
            e_cat = 'Small Aircraft'
          elif msg == 3:
            e_cat = 'Medium Aircraft'
          elif msg == 5:
            e_cat = 'Heavy Aircraft'
          elif msg == 6:
            e_cat = 'High Manoeuvrable Aircraft'
          elif msg == 10:
            e_cat = 'Rotocraft'
          elif msg == 11:
            e_cat = 'Glider'
          elif msg == 12:
            e_cat = 'Lighter than Air Aircraft'
          elif msg == 13:
            e_cat = 'Unmanned Aerial Vehicle'
          elif msg == 14:
            e_cat = 'Space Vehicle'
          else:
            e_cat = 'No Info.'
          E_CAT.append(e_cat)
          #print(f'I020: {e_cat}')
          lugar += 2

      else:
        #print(f'el campo {i+1} es 0')
        campo += 1
        continue
  if fspec_bin[6] == '0':
    LAT.append('NONE')
    LON.append('NONE')
  if fspec_bin[10] == '0':
    T_ADDR.append('NONE')
  if fspec_bin[11] == '0':
    TIME.append('NONE')
  if fspec_bin[15] == '0':
    HEIGHT.append('NONE')
  if fspec_bin[28] == '0':
    T_ID.append('NONE')
  if fspec_bin[29] == '0':
    E_CAT.append('No Info.')


def analisis(hex):
  if leerCategoria(hex) == True:
    CAT.append(21)
    leerLongitud(hex)
    leerFspec(hex)
    analizarFspec(hex,fspec_hex,fspec_b)
    # print(CAT,LONG,LAT,LON,T_ADDR,TIME,HEIGHT,T_ID,E_CAT)
    # print()


################ Lectura ####################

with open(mensajes, "rt") as msgs:

  # data = msgs.readlines()
  # n_msg=len(data)
  # print(n_msg)

  CAT = []
  LONG = []
  FSPEC_HEX = []
  FSPEC_BIN = []
  FSPEC_B = []     #fspec_b es el mismo que _bin  pero sin el byte de Field Extension Indicator
  campos = []

  LAT = []
  LON = []
  T_ADDR = []
  TIME = []
  HEIGHT = []
  T_ID = []
  E_CAT = []

  k = 1

  for linea in msgs:
      categoria = ''
      longitud = ''
      fspec_hex = ''
      fspec_bin = ''
      fspec_b = ''

      analisis(linea)

      if k % 10000 == 0:
          print(round(k/593919*100,2),"% | ",k)

      k += 1

  #Creamos un dicionario con las listas:
  datos = {'Target Addres':T_ADDR,'TimeStamp':TIME,'Lat':LAT,'Lon':LON,'Height':HEIGHT,'Target ID':T_ID,'Emitter Cat.':E_CAT}
  df = pd.DataFrame.from_dict(datos, orient='index')
  df = df.transpose()
  #print(df)
  #Creamos el Excel con nuestro dicionario:
  writer = ExcelWriter("./gdrive/My Drive/TFG/TablasDeDatos/AnalisisHEX_ADSB_08.xlsx")
  df.to_excel(writer, 'Datos ADS-B', index=False)
  writer.save()
  print("100 %")


In [3]:
import pyModeS as pms

Hex = '150044F7193103C3270414D71141000CA8011CC7C3FD76510E63E15CFEBB28BB3440D7384079110101D0023840C4431E608208201508D4015501100606064405086111E8'


CAT = []
LONG = []
FSPEC_HEX = []
FSPEC_BIN = []
FSPEC_B = []     #fspec_b son los mismos que los otros pero sin el byte de Field Extension Indicator
categoria = ''
longitud = ''
fspec_hex = ''
fspec_bin = ''
fspec_b = ''
campos = []

LAT = []
LON = []
T_ADDR = []
TIME = []
HEIGHT = []
T_ID = []
E_CAT = []


def leerCategoria(hex):
  if hex[0:2] == '15':
    return True

def leerLongitud(hex):
  global longitud
  longitud = pms.hex2int(hex[2:6])
  LONG.append(longitud)

def leerFspec(hex):
  global fspec_hex, fspec_h, fspec_bin, fspec_bin2, fspec_b, len_fspec
  contador = list(range(20))
  for i in contador:
    campo_hex = hex[6+i*2:8+i*2]
    fspec_hex += campo_hex
    campo_bin = pms.hex2bin(campo_hex)
    fspec_b = fspec_b + campo_bin[:-1]
    if campo_bin[-1] != '1':
      break
    else:
      continue
  fspec_bin = pms.hex2bin(fspec_hex)
  FSPEC_HEX.append(fspec_hex)
  FSPEC_BIN.append(fspec_bin)
  FSPEC_B.append(fspec_b)



def analizarFspec(hex,fspec_hex,fspec_bin):
  len_fspec = list(range(len(fspec_bin)))
  lugar = len(fspec_hex) + 6
  for i in len_fspec:
    if fspec_bin[i] == '1':
      campo = i + 1
      if campo == 1:
        #print(f'I010: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 2:
        contador = list(range(10))
        for i in contador:
          campo_hex = hex[lugar:lugar+2]
          campo_bin = pms.hex2bin(campo_hex)
          if campo_bin[-1] != '1':
            lugar += 2
            break
          else:
            lugar += 2
            continue
      if campo == 3:
        #print(f'I161: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 4:
        #print(f'I015: {hex[lugar:lugar+2]}')
        lugar += 2
      if campo == 5: 
        #print(f'I071: {hex[lugar:lugar+6]}')
        lugar += 6
      if campo == 6:
        msg = hex[lugar:lugar+12]
        #print(f'I130: {msg}')
        lugar += 12
      if campo == 7:
        msg = hex[lugar:lugar+16]
        s1 = msg[:len(msg)//2]
        s2 = msg[len(msg)//2:]
        lat = int(s1, 16)*(180/(2**30))
        lon = int(s2, 16)*(180/(2**30))
        if lon > 360:
          lon = lon - 360
        LAT.append(lat)
        LON.append(lon)
        print(f'I131: Lat-> {lat}, Lon-> {lon}')
        lugar += 16
      if campo == 8:
        #print(f'I072: {hex[lugar:lugar+6]}')
        lugar += 6
      if campo == 9:
        #print(f'I150: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 10:
        #print(f'I151: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 11:
        t_addr = hex[lugar:lugar+6]
        T_ADDR.append(t_addr)
        print(f'I080: {t_addr}')
        lugar += 6
      if campo == 12:
        msg = hex[lugar:lugar+6]
        time = int(msg, 16)*(1/128)
        TIME.append(time)
        print(f'I073: {time}')
        lugar += 6
      if campo == 13:
        #print(f'I074: {hex[lugar:lugar+8]}')
        lugar += 8
      if campo == 14:
        #print(f'I075: {hex[lugar:lugar+6]}')
        lugar += 6
      if campo == 15:
        #print(f'I076: {hex[lugar:lugar+8]}')
        lugar += 8
      if campo == 16:
        msg = hex[lugar:lugar+4]
        h = int(msg, 16)*6.25
        HEIGHT.append(h)
        print(f'I140: {h}')
        lugar += 4
      if campo == 17:
        contador = list(range(10))
        for i in contador:
          campo_hex = hex[lugar:lugar+2]
          campo_bin = pms.hex2bin(campo_hex)
          if campo_bin[-1] != '1':
            lugar += 2
            break
          else:
            lugar += 2
            continue 
      if campo == 18:
        #print(f'I210: {hex[lugar:lugar+2]}')
        lugar += 2
      if campo == 19:
        #print(f'I070: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 20:
        #print(f'I230: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 21:
        #print(f'I145: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 22:
        #print(f'I152: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 23:
        #print(f'I200: {hex[lugar:lugar+2]}')
        lugar += 2
      if campo == 24:
        #print(f'I155: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 25:
        #print(f'I157: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 26:
        #print(f'I160: {hex[lugar:lugar+8]}')
        lugar += 8
      if campo == 27:
        #print(f'I165: {hex[lugar:lugar+4]}')
        lugar += 4
      if campo == 28:
        #print(f'I077: {hex[lugar:lugar+6]}')
        lugar += 6
      if campo == 29:
        msg = pms.hex2bin(hex[lugar:lugar+12])
        s1 = msg[:len(msg)//8]
        s2 = msg[len(msg)//8:2*len(msg)//8]
        s3 = msg[2*len(msg)//8:3*len(msg)//8]
        s4 = msg[3*len(msg)//8:4*len(msg)//8]
        s5 = msg[4*len(msg)//8:5*len(msg)//8]
        s6 = msg[5*len(msg)//8:6*len(msg)//8]
        s7 = msg[6*len(msg)//8:7*len(msg)//8]
        s8 = msg[7*len(msg)//8:]
        s = [s1,s2,s3,s4,s5,s6,s7,s8]
        t_id = list(range(len(s)))
        for i in list(range(len(s))):
          if s[i] == '000001':
            t_id[i] = 'A'
          elif s[i] == '000010':
            t_id[i] = 'B'
          elif s[i] == '000011':
            t_id[i] = 'C'
          elif s[i] == '000100':
            t_id[i] = 'D'
          elif s[i] == '000101':
            t_id[i] = 'E'
          elif s[i] == '000110':
            t_id[i] = 'F'
          elif s[i] == '000111':
            t_id[i] = 'G'
          elif s[i] == '001000':
            t_id[i] = 'H'
          elif s[i] == '001001':
            t_id[i] = 'I'
          elif s[i] == '001010':
            t_id[i] = 'J'
          elif s[i] == '001011':
            t_id[i] = 'K'
          elif s[i] == '001100':
            t_id[i] = 'L'
          elif s[i] == '001101':
            t_id[i] = 'M'
          elif s[i] == '001110':
            t_id[i] = 'N'
          elif s[i] == '001111':
            t_id[i] = 'O'
          elif s[i] == '010000':
            t_id[i] = 'P'
          elif s[i] == '010001':
            t_id[i] = 'Q'
          elif s[i] == '010010':
            t_id[i] = 'R'
          elif s[i] == '010011':
            t_id[i] = 'S'
          elif s[i] == '010100':
            t_id[i] = 'T'
          elif s[i] == '010101':
            t_id[i] = 'U'
          elif s[i] == '010110':
            t_id[i] = 'V'
          elif s[i] == '010111':
            t_id[i] = 'W'
          elif s[i] == '011000':
            t_id[i] = 'X'
          elif s[i] == '011001':
            t_id[i] = 'Y'
          elif s[i] == '011010':
            t_id[i] = 'Z'
          elif s[i] == '100000':
            t_id[i] = ''
          elif s[i] == '110000':
            t_id[i] = '0'
          elif s[i] == '110001':
            t_id[i] = '1'
          elif s[i] == '110010':
            t_id[i] = '2'
          elif s[i] == '110011':
            t_id[i] = '3'
          elif s[i] == '110100':
            t_id[i] = '4'
          elif s[i] == '110101':
            t_id[i] = '5'
          elif s[i] == '110110':
            t_id[i] = '6'
          elif s[i] == '110111':
            t_id[i] = '7'
          elif s[i] == '111000':
            t_id[i] = '8'
          elif s[i] == '111001':
            t_id[i] = '9'
        t_id2 = "".join(map(str,t_id))
        T_ID.append(str(t_id2))
        print(f'I170: {t_id2}')
        lugar += 12
      if campo == 30:
        msg = int(hex[lugar:lugar+2], 16)
        if msg == 0:
          e_cat = 'No Info.'
        elif msg == 1:
          e_cat = 'Light Aircraft'
        elif msg == 2:
          e_cat = 'Small Aircraft'
        elif msg == 3:
          e_cat = 'Medium Aircraft'
        elif msg == 5:
          e_cat = 'Heavy Aircraft'
        elif msg == 6:
          e_cat = 'High Manoeuvrable Aircraft'
        elif msg == 10:
          e_cat = 'Rotocraft'
        elif msg == 11:
          e_cat = 'Glider'
        elif msg == 12:
          e_cat = 'Lighter than Air Aircraft'
        elif msg == 13:
          e_cat = 'Unmanned Aerial Vehicle'
        elif msg == 14:
          e_cat = 'Space Vehicle'
        else:
          e_cat = 'No Info.'
        E_CAT.append(e_cat)
        print(f'I020: {e_cat}')
        lugar += 2

    else:
      #print(f'el campo {i+1} es 0')
      campo += 1
      continue
  if fspec_bin[6] == '0':
    LAT.append('NONE')
    LON.append('NONE')
  if fspec_bin[10] == '0':
    T_ADDR.append('NONE')
  if fspec_bin[11] == '0':
    TIME.append('NONE')
  if fspec_bin[15] == '0':
    HEIGHT.append('NONE')
  if fspec_bin[28] == '0':
    T_ID.append('NONE')
  if fspec_bin[29] == '0':
    E_CAT.append('No Info.')


def analisis(hex):
  if leerCategoria(hex) == True:
    CAT.append(21)
    leerLongitud(hex)
    leerFspec(hex)
    analizarFspec(hex,fspec_hex,fspec_b)

  print(FSPEC_B)
  print(CAT,LONG,LAT,LON,T_ADDR,TIME,HEIGHT,T_ID,E_CAT)

analisis(Hex)

I131: Lat-> 40.47231785953045, Lon-> 356.4311913214624
I080: 3440D7
I073: 28800.9453125
I170: P19
I020: No Info.
['1111011000110000110000000001110000100100110000010']
[21] [68] [40.47231785953045] [356.4311913214624] ['3440D7'] [28800.9453125] ['NONE'] ['P19'] ['No Info.']
